In [1]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score

## Loading and Prepping data

In [2]:
data = pd.read_csv('./data/train_data1.csv')

In [3]:
data.tail()

,domain,domain_type
199995,shao-shavers.myshopify.com,benign
199996,aissf.in,benign
199997,bossofthefloss.blogspot.com,benign
199998,symextech.com,benign
199999,kartell.com,benign


In [15]:
# sample_data = data.sample(50000)

In [16]:
# sample_data.groupby('domain_type').count()

,domain
domain_type,
benign,24965
dga,25035


In [4]:
# sample_data['target'] = sample_data.domain_type.apply(lambda x: 0 if x=='benign' else 1)
data['target'] = data.domain_type.apply(lambda x: 0 if x=='benign' else 1)

In [5]:
# sample_data.head()
data.head()

,domain,domain_type,target
0,ofdhiydrrttpblp.com,dga,1
1,puciftnfkplcbhp.net,dga,1
2,bowjjxxnhkyvygk.biz,dga,1
3,osvwkptpwqyiqen.ru,dga,1
4,cpmjpnwdgbxyyql.org,dga,1


## Feature Engineering
All we have is a string and for any model having multiple features is important. Thus, in this section we will try to create features from the name of our domains. Since XGBoost needs numeric features, we need to encode the characters. I have created a dictionary of our character mappings.

In [6]:
##We might use it later
def ngrams(n, arr):
    ''' Creates ngram mappings and their counts of a given list of strings
        Returns a dictionary
    '''
    freq = {}
    for s in arr:
        for i in range(len(s)-2):
            key = s[i:i+3]
            if key in freq:
                freq[key]+=1
            else:
                freq[key]=1
    return freq

In [7]:
##Testing the above function
ngrams(3, ['facebook', 'uk'])

{'fac': 1, 'ace': 1, 'ceb': 1, 'ebo': 1, 'boo': 1, 'ook': 1}

In [8]:
data['domain_list'] = data.domain.apply(lambda x: ngrams(3, x.split('.')))

In [9]:
data['length'] = data.domain.apply(lambda x: len(x))
data.head()

,domain,domain_type,target,domain_list,length
0,ofdhiydrrttpblp.com,dga,1,"{'ofd': 1, 'fdh': 1, 'dhi': 1, 'hiy': 1, 'iyd'...",19
1,puciftnfkplcbhp.net,dga,1,"{'puc': 1, 'uci': 1, 'cif': 1, 'ift': 1, 'ftn'...",19
2,bowjjxxnhkyvygk.biz,dga,1,"{'bow': 1, 'owj': 1, 'wjj': 1, 'jjx': 1, 'jxx'...",19
3,osvwkptpwqyiqen.ru,dga,1,"{'osv': 1, 'svw': 1, 'vwk': 1, 'wkp': 1, 'kpt'...",18
4,cpmjpnwdgbxyyql.org,dga,1,"{'cpm': 1, 'pmj': 1, 'mjp': 1, 'jpn': 1, 'pnw'...",19


In [10]:
##Encoding the characters:
char_map = {chr(i+97): i for i in range(26)} ##For all the alphabets
char_map['.'] = 27
char_map['_'] = 28
char_map['-'] = 29

In [11]:
count = 30
##For all the digits
for i in range(10):
    char_map[str(i)] = count
    count+=1

In [12]:
##Padding to keep the vectors of same length
char_map[' '] = 40 

In [14]:
max_length = max(data.length) ##length of our embedding

In [15]:
##Creates encoded vector 
def encodeDomain(domain, length):
    vector = np.full(shape= length, fill_value=29)
    for i in range(len(domain)):
        if domain[i] not in char_map:
            print('Invalid key ', domain[i])
        else:
            vector[i] = char_map[domain[i]]
    return vector

In [16]:
##Testing above function
encodeDomain('facebook.com', 40)

array([ 5,  0,  2,  4,  1, 14, 14, 10, 27,  2, 14, 12, 29, 29, 29, 29, 29,
       29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29,
       29, 29, 29, 29, 29, 29])

In [17]:
embedding = data.domain.apply(lambda x : encodeDomain(x, max_length))

## Model 1: With just the encodings

In [19]:
# split data into train and test sets
seed = 7
test_size = 0.33
X = np.array(embedding.tolist())
y = np.array(data.target)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed)

In [20]:
##Training the model
model = XGBClassifier()
model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
       importance_type='gain', interaction_constraints=None,
       learning_rate=0.300000012, max_delta_step=0, max_depth=6,
       min_child_weight=1, missing=nan, monotone_constraints=None,
       n_estimators=100, n_jobs=0, num_parallel_tree=1,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method=None,
       validate_parameters=False, verbosity=None)

In [21]:
##Testing the model
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [22]:
#Evaluating predictions
accuracy = accuracy_score(y_test, predictions)
f_score = f1_score(y_test, predictions)
print("Accuracy: %.3f%%" % (accuracy * 100.0))
print("F1_score: ", f_score)

Accuracy: 98.936%
F1_score:  0.9894194248507868


The accuracy and F1-score are both really high. Hence, our first version of the model is doing good.

## Predicting the given test data

In [23]:
grading_data = pd.read_csv('./data/grading_data.csv')

In [29]:
print(len(grading_data))
grading_data.head()

500


,domain
0,www.dutchwebdesign.com
1,www.pploulsosohvmkv.com
2,www.cfsjdqwdhmwkiv.com
3,www.uocoqgiusyeciouaimcauykqswsymo.com
4,www.total-toolbar.com


In [25]:
grade_embedding = grading_data.domain.apply(lambda x : encodeDomain(x, max_length))

In [27]:
X_grade = np.array(grade_embedding.tolist())

In [28]:
grade_pred = model.predict(X_grade)
predictions_grade = [round(value) for value in grade_pred]

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [31]:
grading_data['threat'] = ['benign' if pred==0 else 'dga' for pred in predictions_grade]

In [33]:
grading_data.head()

,domain,threat
0,www.dutchwebdesign.com,benign
1,www.pploulsosohvmkv.com,benign
2,www.cfsjdqwdhmwkiv.com,benign
3,www.uocoqgiusyeciouaimcauykqswsymo.com,benign
4,www.total-toolbar.com,benign


In [35]:
grading_data.to_csv('predictions1.csv', index=False)